In [5]:
import pandas as pd 
import numpy as np 
import datetime 


from CV import cross_validation as CV
from CV import combinatorial as CB

# Order 
1. X, Y generation 
2. Train(valid) / Test splitting
3. CPCV
   어차피 여기서 뒤에서 개수만큼 잘라주는 거면 데이터 포인트를 자르는 것과 다를바 없음
   


In [6]:
df = pd.read_csv("./data/data_input_demo.csv", index_col = [0])
df = df.set_index(['date'])

In [7]:
df.head()

,13ty_index,interty_index,lty_index,mbs_index,13cy_index,intercy_index,lcy_index,ty_index,cy_index,agg_index,real_known,cat_obs1,cat_obs2,cat_knwon1,cat_knwon2,static
date,,,,,,,,,,,,,,,,
1997-05-19,133.46,784.22,840.64,751.37,668.10,893.58,912.47,824.23,861.00,715.66,215.632652,2,2,4,3,1.0
1997-05-20,133.58,785.01,840.34,751.82,668.74,894.56,912.29,824.78,861.51,716.09,980.112727,2,1,5,4,1.0
1997-05-21,133.59,784.63,836.47,751.97,668.77,893.58,908.76,823.54,859.71,715.31,247.766286,2,1,4,4,1.0
1997-05-22,133.58,784.33,834.66,751.97,668.71,892.96,906.86,822.86,858.69,714.88,306.038065,2,1,3,4,1.0
1997-05-23,133.58,784.70,835.59,752.27,668.71,893.58,907.77,823.37,859.37,715.31,944.644883,0,2,3,3,1.0


# X, Y generation 

In [8]:

def generate_xy_seq(df: pd.DataFrame, x_seq = 66, y_seq = 22):
    """
    Generate samples from
    :param df:
    :param x_seq:
    :param y_seq:
    :param scaler:
    :return:
    # x: (epoch_size, input_length, num_nodes, input_dim)
    # y: (epoch_size, output_length, num_nodes, output_dim)
    """
    num_samples, num_nodes = df.shape
    dates_arr = np.array(df.index)
    data = np.expand_dims(df.values, axis = -1) # df -> array [N, F, 1]

    x_offsets = np.arange(-x_seq+1, 1)  
    y_offsets = np.arange(1, y_seq+1)

    # feature_list = [data]

    x, y = [], []
    x_date, y_date = [],[]

    min_t = abs(min(x_offsets))
    max_t = abs(num_samples - abs(max(y_offsets)))

    for t in range(min_t, max_t):
        # value seperation
        x.append(data[t+x_offsets, ...])
        y.append(data[t+y_offsets, ...])
        # date seperation
        x_date.append(dates_arr[t+x_offsets])
        y_date.append(dates_arr[t+y_offsets])
        
    x = np.stack(x, axis = 0)
    y = np.stack(y, axis = 0)

    x_date = np.stack(x_date, axis = 0)
    y_date = np.stack(y_date, axis = 0)

    return x, y, x_date, y_date

In [9]:
x, y, x_date, y_date = generate_xy_seq(df)

In [10]:
x.shape

(6298, 66, 16, 1)

In [11]:
x_date.shape

(6298, 66)

# Splitting

In [12]:
num_samples = df.shape[0]
num_train = round(num_samples * 0.8)
# num_val = round(num_train * 0.2)
# num_train = num_train - num_val   
num_test = num_samples - (num_train)

In [13]:
TRAIN = df.iloc[:num_train]
TEST = df.iloc[num_train : ]

# CPCV
- 6C2
- purging
- embargo 

In [14]:
from itertools import combinations

In [15]:
numFold = 6 
combNum = 2
folds = [i for i in range(numFold)]
val_comb = list(combinations(folds, combNum))
fold_set = [(ix[0], ix[-1] + 1) for ix in np.array_split(np.arange(TRAIN.shape[0]), 6)]

In [16]:
val_comb

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 4),
 (3, 5),
 (4, 5)]

In [17]:
fold_set

[(0, 852), (852, 1704), (1704, 2555), (2555, 3406), (3406, 4257), (4257, 5108)]

In [18]:
a = (0,2)
if 0 in a:
    print ("TRUE")

TRUE


In [41]:
comb

(4, 5)

In [19]:
cv_set = {}

# for comb in val_comb: # (0,1)

# for one combination 
comb = val_comb[0]
valid_set = {}
train_set = {}

for fold_ind, _ in enumerate(fold_set):
    
    if fold_ind in comb: 
        fold = fold_set[fold_ind]        
        
        for (st_ind, end_ind) in [fold]:
            valid = TRAIN.iloc[st_ind : end_ind]
            valid_set[fold_ind] = (valid)
    else:
        
        fold = fold_set[fold_ind]
        for st_ind, end_ind in [fold]:
            train = TRAIN.iloc[st_ind : end_ind]
            train_set[fold_ind] = (train)
                
cv_set['val'] = valid_set
cv_set['train'] = train_set

In [46]:
a = {'d':[1,2,3],'e':[2,4,5]}
a.values()

dict_values([[1, 2, 3], [2, 4, 5]])

In [45]:
cv_set['val'].values()

dict_values([            13ty_index  interty_index  lty_index  mbs_index  13cy_index  \
date                                                                      
2010-07-08      243.85        1619.92    2334.34    1715.26     1355.03   
2010-07-09      243.76        1618.02    2322.43    1715.67     1354.92   
2010-07-12      243.75        1618.93    2324.04    1716.79     1355.34   
2010-07-13      243.65        1615.57    2305.16    1715.58     1355.31   
2010-07-14      243.92        1620.02    2327.30    1717.80     1357.27   
...                ...            ...        ...        ...         ...   
2013-09-04      249.76        1730.50    2699.53    1825.08     1461.70   
2013-09-05      249.53        1725.60    2664.88    1815.33     1460.01   
2013-09-06      249.74        1729.04    2669.89    1821.96     1461.57   
2013-09-09      249.91        1732.40    2685.27    1829.28     1462.70   
2013-09-10      249.77        1729.44    2662.95    1824.41     1461.85   

           

test 가 위치할 수 있는 경우
purging

1. train | test : test 에서 버리기 
   train[-1], test[0] 랑 비교 


2. test | train : embargo 좀 많이 버리기 from train 에서 

   test[-1], train[0] 랑 비교




In [20]:
purging_num = 22
embargo_num = 22

In [21]:
def purging(cv_set):
    '''
    cv_set (dict): keys (train, val), values
    '''
    val_keys = list(cv_set['val'].keys())
    train_keys = list(cv_set['train'].keys())
    # print ("val_keys ", val_keys)
    for val_k in val_keys:
        print (val_k)
        if val_k-1 in train_keys: # train | test -> test 앞 버리기 
            print (" o | x ")
            cv_set['val'][val_k] = cv_set['val'][val_k].iloc[purging_num:]
            if val_k+1 in train_keys: # test | train -> train 앞 버리기 
                print (" x | o ")
                cv_set['train'][val_k+1] = cv_set['train'][val_k+1].iloc[embargo_num : ]

        elif val_k+1 in train_keys: # test | train -> train 앞 버리기 
            print (" x | o ")
            cv_set['train'][val_k+1] = cv_set['train'][val_k+1].iloc[embargo_num : ]
            if val_k-1 in train_keys: # train | test -> test 앞 버리기 
                print (" o | x | ")
                cv_set['val'][val_k] = cv_set['val'][val_k].iloc[purging_num:]
    return cv_set

In [36]:
valid_set.keys()

dict_keys([4, 5])

In [37]:
fold

(4257, 5108)

In [38]:
fold_set

[(0, 852), (852, 1704), (1704, 2555), (2555, 3406), (3406, 4257), (4257, 5108)]

In [39]:
val_comb

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 4),
 (3, 5),
 (4, 5)]

In [40]:
fold_set

[(0, 852), (852, 1704), (1704, 2555), (2555, 3406), (3406, 4257), (4257, 5108)]

In [32]:

CPCV = []
for comb in val_comb: # (0,1)
    print (f"{comb}-----------")
    cv_set = {}

    # for one combination 
    valid_set = {}
    train_set = {}

    for fold_ind, _ in enumerate(fold_set):
    # for fold_ind in list(comb):
        # print ("fold_ind: ", fold_ind)
        if fold_ind in comb: 
            # print ("VALD")
            fold = fold_set[fold_ind]        
            # print (fold)
            for (st_ind, end_ind) in [fold]:
                valid = TRAIN.iloc[st_ind : end_ind]
                valid_set[fold_ind] = valid
        else:
            # print ("TRAIN ")
            fold = fold_set[fold_ind]
            for st_ind, end_ind in [fold]:
                # print ('st_ind: ', st_ind)
                train = TRAIN.iloc[st_ind : end_ind]
                train_set[fold_ind] = train
                    
    cv_set['val'] = valid_set
    cv_set['train'] = train_set
    # print (cv_set)
    cv_set = purging(cv_set)
    
    CPCV.append(cv_set)

(0, 1)-----------
{'val': {0:             13ty_index  interty_index  lty_index  mbs_index  13cy_index  \
date                                                                      
1997-05-19      133.46         784.22     840.64     751.37      668.10   
1997-05-20      133.58         785.01     840.34     751.82      668.74   
1997-05-21      133.59         784.63     836.47     751.97      668.77   
1997-05-22      133.58         784.33     834.66     751.97      668.71   
1997-05-23      133.58         784.70     835.59     752.27      668.71   
...                ...            ...        ...        ...         ...   
2000-08-16      160.03         950.72    1124.45     924.39      810.25   
2000-08-17      160.08         951.02    1127.72     924.66      810.48   
2000-08-18      160.16         952.13    1131.18     925.38      810.92   
2000-08-21      160.22         952.41    1129.04     925.35      811.15   
2000-08-22      160.31         953.07    1129.72     926.32      811.7

In [23]:
len(CPCV)

15

In [24]:
CPCV[10]['val'].keys()

dict_keys([2, 4])

In [25]:
CPCV[0]['val'][0]

,13ty_index,interty_index,lty_index,mbs_index,13cy_index,intercy_index,lcy_index,ty_index,cy_index,agg_index,real_known,cat_obs1,cat_obs2,cat_knwon1,cat_knwon2,static
date,,,,,,,,,,,,,,,,
1997-05-19,133.46,784.22,840.64,751.37,668.10,893.58,912.47,824.23,861.00,715.66,215.632652,2,2,4,3,1.0
1997-05-20,133.58,785.01,840.34,751.82,668.74,894.56,912.29,824.78,861.51,716.09,980.112727,2,1,5,4,1.0
1997-05-21,133.59,784.63,836.47,751.97,668.77,893.58,908.76,823.54,859.71,715.31,247.766286,2,1,4,4,1.0
1997-05-22,133.58,784.33,834.66,751.97,668.71,892.96,906.86,822.86,858.69,714.88,306.038065,2,1,3,4,1.0
1997-05-23,133.58,784.70,835.59,752.27,668.71,893.58,907.77,823.37,859.37,715.31,944.644883,0,2,3,3,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2000-08-16,160.03,950.72,1124.45,924.39,810.25,1080.00,1115.47,1026.86,1045.26,881.94,226.026624,2,2,5,5,1.0
2000-08-17,160.08,951.02,1127.72,924.66,810.48,1080.27,1116.19,1028.11,1045.64,882.45,635.443038,2,1,4,3,1.0
2000-08-18,160.16,952.13,1131.18,925.38,810.92,1081.80,1118.81,1029.99,1047.40,883.65,776.453337,0,1,4,4,1.0


In [26]:
CPCV[0]['val'][1]

,13ty_index,interty_index,lty_index,mbs_index,13cy_index,intercy_index,lcy_index,ty_index,cy_index,agg_index,real_known,cat_obs1,cat_obs2,cat_knwon1,cat_knwon2,static
date,,,,,,,,,,,,,,,,
2000-08-23,160.45,954.75,1134.57,928.52,812.58,1085.14,1119.12,1032.91,1049.78,886.27,33.575711,1,2,4,5,1.0
2000-08-24,160.53,955.27,1136.50,929.04,813.02,1085.78,1120.49,1033.88,1050.59,886.99,512.443063,1,2,5,5,1.0
2000-08-25,160.55,955.43,1136.38,928.72,813.15,1086.21,1120.54,1033.96,1050.90,886.97,159.283896,2,0,3,3,1.0
2000-08-28,160.56,954.74,1130.61,927.80,813.45,1085.17,1116.08,1031.65,1048.97,885.59,523.706450,0,2,5,3,1.0
2000-08-29,160.51,953.89,1127.21,926.35,813.13,1084.03,1112.45,1029.97,1047.20,884.15,7.321231,1,0,3,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2003-11-21,193.90,1205.67,1507.75,1174.57,1023.41,1459.11,1594.77,1326.99,1430.36,1153.90,688.199348,1,0,4,3,1.0
2003-11-24,193.69,1202.62,1496.95,1172.61,1022.13,1454.74,1584.96,1321.79,1424.99,1150.49,628.563981,0,1,4,4,1.0
2003-11-25,193.81,1204.16,1502.98,1175.07,1022.95,1457.12,1590.05,1324.57,1427.86,1152.81,56.933194,1,2,5,4,1.0


In [27]:
CPCV[0]['train'][2]

,13ty_index,interty_index,lty_index,mbs_index,13cy_index,intercy_index,lcy_index,ty_index,cy_index,agg_index,real_known,cat_obs1,cat_obs2,cat_knwon1,cat_knwon2,static
date,,,,,,,,,,,,,,,,
2003-12-30,194.53,1207.15,1501.30,1181.11,1027.96,1464.05,1599.64,1326.42,1435.05,1157.38,845.028169,2,0,5,5,1.0
2003-12-31,194.57,1207.86,1502.86,1182.64,1028.29,1465.06,1600.80,1327.38,1436.06,1158.44,594.303874,1,1,5,3,1.0
2004-01-01,194.57,1207.86,1502.86,1182.64,1028.29,1465.06,1600.80,1327.38,1436.06,1158.44,65.824746,2,2,3,3,1.0
2004-01-02,194.25,1203.57,1486.59,1178.78,1026.62,1459.13,1586.13,1319.84,1428.52,1153.28,654.858079,0,0,5,3,1.0
2004-01-05,194.28,1203.42,1486.60,1179.72,1026.85,1459.37,1589.03,1319.72,1429.31,1153.80,293.528200,0,2,3,5,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-02-26,209.01,1307.72,1780.40,1351.78,1126.70,1637.15,1927.34,1471.39,1632.36,1307.01,641.290990,0,1,4,4,1.0
2007-02-27,209.58,1314.06,1800.82,1355.87,1129.91,1645.57,1942.98,1480.55,1641.98,1313.21,770.337219,2,2,5,4,1.0
2007-02-28,209.47,1312.50,1794.08,1354.35,1129.54,1643.20,1935.23,1478.00,1638.56,1311.09,425.250213,1,1,3,5,1.0


In [28]:
CPCV[10]['val'].keys()

dict_keys([2, 4])

In [29]:
CPCV[10]['train'].keys()

dict_keys([0, 1, 3, 5])

In [30]:
CPCV[10]['val']

{2:             13ty_index  interty_index  lty_index  mbs_index  13cy_index  \
 date                                                                      
 2003-12-30      194.53        1207.15    1501.30    1181.11     1027.96   
 2003-12-31      194.57        1207.86    1502.86    1182.64     1028.29   
 2004-01-01      194.57        1207.86    1502.86    1182.64     1028.29   
 2004-01-02      194.25        1203.57    1486.59    1178.78     1026.62   
 2004-01-05      194.28        1203.42    1486.60    1179.72     1026.85   
 ...                ...            ...        ...        ...         ...   
 2007-02-26      209.01        1307.72    1780.40    1351.78     1126.70   
 2007-02-27      209.58        1314.06    1800.82    1355.87     1129.91   
 2007-02-28      209.47        1312.50    1794.08    1354.35     1129.54   
 2007-03-01      209.55        1312.87    1792.54    1354.32     1129.89   
 2007-03-02      209.79        1315.14    1798.71    1355.30     1131.08   
 
       

## X, Y generation 

In [31]:
def purging_data(samples_info_sets: pd.Series,
                 test_times: pd.Series) -> pd.Series:
    '''
    제가 기존에 사용하던 데이터는 학습 데이터와 라벨 사이의 길이가 전부 다르고 동시에 발생하기도 합니다,
    때문에 데이터를 split할때 infomation leakage가 발생하지 않기 위해 매우 신경을 써야 했습니다.
    Train starts within test & # Train ends within test & # Train envelops test를 drop해서 사용합니다
    이후 Cross Validation을 할때도 사용되게 됩니다
    CV시에는 추가적으로 테스트 셋 뒤쪽에 embargo도 넣어줍니다

    '''
    train = samples_info_sets.copy(deep=True)
    for start_ix, end_ix in test_times.iteritems():
        df0 = train[(start_ix <= train.index) & (train.index <= end_ix)].index  # Train starts within test
        df1 = train[(start_ix <= train) & (train <= end_ix)].index  # Train ends within test
        df2 = train[(train.index <= start_ix) & (end_ix <= train)].index  # Train envelops test
        train = train.drop(df0.union(df1).union(df2))
    return train


class PurgedKFold():
    '''
    Purging 과 Embargo를 통해 K fold CV를 하면서도 information leakage를 막을 수 있습니다.
    '''

    def __init__(self,
                 n_splits: int = 3,
                 samples_info_sets: pd.Series = None,
                 pct_embargo: float = 0.):
        self.n_splits = n_splits
        self.samples_info_sets = samples_info_sets
        self.pct_embargo = pct_embargo

    def split(self,
              X: pd.DataFrame,
              y: pd.Series = None):

        if X.shape[0] != self.samples_info_sets.shape[0]:
            raise ValueError("X 와 'samples_info_sets' 의 길이가 다릅니다")

        indices: np.ndarray = np.arange(X.shape[0])
        embargo: int = int(X.shape[0] * self.pct_embargo)

        test_ranges: [(int, int)] = [(ix[0], ix[-1] + 1) for ix in np.array_split(np.arange(X.shape[0]), self.n_splits)]
        for start_ix, end_ix in test_ranges:
            test_indices = indices[start_ix:end_ix]
            if end_ix < X.shape[0]:
                end_ix += embargo
            test_times = pd.Series(index=[self.samples_info_sets[start_ix]], data=[self.samples_info_sets[end_ix-1]])
            train_times = purging_data(self.samples_info_sets, test_times)

            train_indices = []
            for train_ix in train_times.index:
                train_indices.append(self.samples_info_sets.index.get_loc(train_ix))
            yield np.array(train_indices), test_indices